#PROTOTIPO 1

In [1]:
import os
import cv2
import numpy as np
import joblib
from deepface import DeepFace
#from deepface.commons import functions

from time import time

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import time

print("Cargando modelo FaceNet...")
model_name = "Facenet"
model = DeepFace.build_model(model_name)
# Obtenemos la dimensión de entrada que espera el modelo (ej. 160x160)
dim = model.input_shape
print(f"Modelo cargado. Tamaño de entrada esperado: {dim}")



Cargando modelo FaceNet...
Modelo cargado. Tamaño de entrada esperado: (160, 160)


In [9]:
# --- 2. FUNCIÓN DE CARGA DEL DATASET (VERSIÓN NUMÉRICA) ---

def LoadDataset_Emociones(folder, dim, max_images_per_class=500):
    nclasses = 0
    nperclass = []
    classlabels = []
    X = []
    Y = []

    # DEFINIMOS LAS CARPETAS QUE QUEREMOS CARGAR (USANDO LOS NÚMEROS)
    # 0=Angry, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
    # Excluimos 1 (Disgust) y 2 (Fear) para simplificar
    target_folders = ['0', '3', '4', '5', '6']

    # Leemos todas las carpetas del disco
    all_folders = sorted([d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))])
    
    # FILTRAMOS: Mantenemos solo las carpetas de nuestra lista
    emotion_folders = [d for d in all_folders if d in target_folders]
    
    print(f"Analizando SOLO estas carpetas (clases): {emotion_folders}")
    print(f"ATENCIÓN: Se cargarán MÁXIMO {max_images_per_class} imágenes por clase.")

    for class_name in emotion_folders:
        print(f"Procesando la carpeta: {class_name} (Clase N.{nclasses})")
        
        class_folder = os.path.join(folder, class_name)
        nsamples = 0
        
        for file_name in os.listdir(class_folder):
            if nsamples >= max_images_per_class:
                break 

            if file_name.endswith('.jpg') or file_name.endswith('.png'):
                image_path = os.path.join(class_folder, file_name)
                image = cv2.imread(image_path)
                if image is None: continue
                
                img_resized = cv2.resize(image, (dim[1], dim[0]), interpolation = cv2.INTER_AREA)

                try:
                    embedding_objs = DeepFace.represent(
                        img_path = img_resized,
                        model_name = model_name, 
                        enforce_detection = False
                    )
                    img_embedding = embedding_objs[0]["embedding"]
                    X.append(img_embedding)
                    Y.append(nclasses)
                    nsamples += 1
                except Exception as e:
                    pass
        
        nperclass.append(nsamples)
        classlabels.append(class_name) # Aquí guardará '0', '3', etc.
        nclasses += 1 
        print(f"-> Encontradas {nsamples} imágenes.")

    X = np.array(X, dtype='float32')
    Y = np.array(Y, dtype='float64')

    return X, Y, classlabels

In [10]:
# --- 3. CARGA DE DATOS Y ENTRENAMIENTO ---

folder_dataset = "C:/dataset_VC" 

path_train = os.path.join(folder_dataset, "train")
path_test = os.path.join(folder_dataset, "test")

# --- Carga de Datos ---
# Este es el paso lento. Carga las imágenes de training y calcula los embeddings.
print("Iniciando carga del dataset de TRAINING...")
t0 = time.time()
# La variable 'dim' fue definida en la Celda 1 (model.input_shape)
X_train, y_train, classlabels = LoadDataset_Emociones(path_train, dim)
print(f"Dataset de Training cargado en {time.time() - t0:.2f} segundos.")

# Carga las imágenes de test y calcula los embeddings.
print("\nIniciando carga del dataset de TEST...")
t0 = time.time()
X_test, y_test, _ = LoadDataset_Emociones(path_test, dim)
print(f"Dataset de Test cargado en {time.time() - t0:.2f} segundos.")

print("\n--- Resumen de Datos ---")
print(f"Clases a entrenar: {classlabels}")
print(f"Muestras de Training: {X_train.shape[0]}, Features: {X_train.shape[1]}")
print(f"Muestras de Test: {X_test.shape[0]}")

# --- Entrenamiento Modelo SVM ---
# Ahora entrenamos nuestro clasificador (como en el ejemplo del profe)
# Usamos un SVM simple en lugar de un GridSearch para ir más rápido.
# Para un mejor resultado, puedes reemplazarlo con un GridSearchCV.

print("\nIniciando entrenamiento del clasificador SVM con Kernel RBF...")
t0 = time.time()

# Usamos kernel='rbf' 
# Usamos C=10 (le damos libertad al modelo para ajustarse más a los datos)
# Usamos gamma='scale' (ajuste automático basado en la varianza de los datos)
model_svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, class_weight='balanced')

# Entrenamos el modelo
model_svm.fit(X_train, y_train)
print(f"Entrenamiento SVM (RBF) completado en {time.time() - t0:.2f} segundos.")

# --- Evaluación del Nuevo Modelo ---
print("\nEvaluación del modelo RBF sobre el set de Test...")
y_pred = model_svm.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"-> Nueva Precisión (Accuracy) en el Test Set: {acc * 100:.2f}%")

# --- GUARDADO ---
model_filename = "mio_modello_emozioni.pkl"
print(f"\nSobrescribiendo el modelo en '{model_filename}'...")
joblib.dump(model_svm, model_filename)
# Las etiquetas no cambian, pero las guardamos por si acaso
joblib.dump(classlabels, "etichette_emozioni.pkl") 

print("--- ENTRENAMIENTO COMPLETADO ---")
print(f"Tu modelo ('{model_filename}') está listo para el Prototipo 1.")

Iniciando carga del dataset de TRAINING...
Analizando SOLO estas carpetas (clases): ['0', '3', '4', '5', '6']
ATENCIÓN: Se cargarán MÁXIMO 500 imágenes por clase.
Procesando la carpeta: 0 (Clase N.0)
-> Encontradas 500 imágenes.
Procesando la carpeta: 3 (Clase N.1)
-> Encontradas 500 imágenes.
Procesando la carpeta: 4 (Clase N.2)
-> Encontradas 500 imágenes.
Procesando la carpeta: 5 (Clase N.3)
-> Encontradas 500 imágenes.
Procesando la carpeta: 6 (Clase N.4)
-> Encontradas 500 imágenes.
Dataset de Training cargado en 1602.18 segundos.

Iniciando carga del dataset de TEST...
Analizando SOLO estas carpetas (clases): ['0', '3', '4', '5', '6']
ATENCIÓN: Se cargarán MÁXIMO 500 imágenes por clase.
Procesando la carpeta: 0 (Clase N.0)
-> Encontradas 491 imágenes.
Procesando la carpeta: 3 (Clase N.1)
-> Encontradas 500 imágenes.
Procesando la carpeta: 4 (Clase N.2)
-> Encontradas 500 imágenes.
Procesando la carpeta: 5 (Clase N.3)
-> Encontradas 416 imágenes.
Procesando la carpeta: 6 (Clase N.

In [13]:
# --- 4. PROTOTIPO LIVE (última celda del notebook) ---

print("Iniciando prototipo en vivo...")
print("Usando el modelo 'model_svm' y las 'classlabels' que acabamos de entrenar.")
print("Presiona 'ESC' en la ventana de la cámara para salir.")


model_svm = joblib.load("mio_modello_emozioni.pkl")
classlabels = joblib.load("etichette_emozioni.pkl")
print(f"✅ Modelo cargado. El modelo conoce {len(classlabels)} clases.") 

# Mappiamo il NOME DELLA CARTELLA (stringa) all'EMOZIONE
MAPA_CARPETAS = {
    '0': 'angry',
    '1': 'disgust',
    '2': 'fear',
    '3': 'happy',
    '4': 'sad',
    '5': 'surprise',
    '6': 'neutral'
}

COLORES_REACCION = {
    'angry': (0, 0, 255),    
    'happy': (0, 255, 255),  
    'sad': (255, 0, 0),      
    'surprise': (255, 165, 0),
    'neutral': (255, 255, 255),
    'fear': (130, 0, 75),
    'disgust': (0, 100, 0)
}

# --- 3. WEBCAM ---
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: No se puede abrir la cámara.")
else:
    print("Cámara activa. Pulsa 'ESC' para salir.")
    while True:
        ret, frame = cap.read()
        if not ret: break

        frame_para_mostrar = frame.copy()

        try:
            # --- Detección ---
            # Usiamo opencv perché ssd è lento e causa i messaggi "Face not detected"
            embedding_obj = DeepFace.represent(
                img_path = frame, 
                model_name = "Facenet",
                enforce_detection = True,
                detector_backend = "opencv" 
            )
            
            embedding = embedding_obj[0]["embedding"]
            
            # --- Predicción ---
            probabilidades = model_svm.predict_proba([embedding])[0]
            indice_max = np.argmax(probabilidades)
            confianza = probabilidades[indice_max]
            
            # --- LOGICA SICURA (MODIFICA PRINCIPALE) ---
            if confianza < 0.30:
                emocion_real = 'neutral'
                texto_extra = f" (? {int(confianza*100)}%)"
            else:
                # 1. Chiediamo a classlabels qual è il nome della cartella per questo indice
                # Questo non fallisce mai perché classlabels corrisponde al modello
                nombre_carpeta = classlabels[indice_max]
                
                # 2. Traduciamo il nome cartella in emozione
                emocion_real = MAPA_CARPETAS.get(nombre_carpeta, 'neutral')
                
                texto_extra = f" ({int(confianza*100)}%)"

            # --- Reazioni ---
            area = embedding_obj[0]["facial_area"]
            x, y, w, h = area['x'], area['y'], area['w'], area['h']
            color = COLORES_REACCION.get(emocion_real, (128, 128, 128))

            texto_reaccion = emocion_real.upper() + texto_extra
            
            cv2.rectangle(frame_para_mostrar, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame_para_mostrar, texto_reaccion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Filtri
            overlay = frame_para_mostrar.copy()
            apply_overlay = False
            color_fondo = (0, 0, 0) # Default

            if emocion_real == 'happy':
                color_fondo = (0, 255, 255) # Amarillo
                apply_overlay = True
            
            elif emocion_real == 'sad':
                color_fondo = (255, 0, 0)   # Azul (tristeza)
                apply_overlay = True
            
            elif emocion_real == 'angry':
                color_fondo = (0, 0, 255)   # Rojo
                apply_overlay = True
            
            elif emocion_real == 'surprise':
                color_fondo = (255, 0, 0) # Naranja
                apply_overlay = True

            # Aplicamos el filtro si corresponde
            if apply_overlay:
                # 1. Dibuja un rectángulo lleno (-1) sobre toda la pantalla
                cv2.rectangle(overlay, (0, 0), (frame.shape[1], frame.shape[0]), color_fondo, -1)
                # 2. Mezcla: 20% color (0.2) + 80% imagen original (0.8)
                frame_para_mostrar = cv2.addWeighted(overlay, 0.2, frame_para_mostrar, 0.8, 0)
        except Exception:
            # Non stampiamo nulla qui per evitare spam in console
            pass

        cv2.imshow('Prototipo 1', frame_para_mostrar)

        if cv2.waitKey(1) & 0xFF == 27: # ESC
            break

    cap.release()
    cv2.destroyAllWindows()
    for i in range(5): cv2.waitKey(1)

Iniciando prototipo en vivo...
Usando el modelo 'model_svm' y las 'classlabels' que acabamos de entrenar.
Presiona 'ESC' en la ventana de la cámara para salir.
✅ Modelo cargado. El modelo conoce 5 clases.
Cámara activa. Pulsa 'ESC' para salir.
